In [1]:
import pandas as pd

# CSV 파일 로드
df = pd.read_csv('표준코드/표준코드_산지코드.csv', encoding='cp949')

# 숫자 비교를 위해 'plor_cd' 정수형으로 변환
df['plor_cd'] = pd.to_numeric(df['plor_cd'], errors='coerce')

# 제외할 지역명 접두어
excluded_prefixes = ['서울', '대전', '세종', '인천', '광주', '부산', '울산', '대구', '제주']

# 제외 조건: plor_cd < 800000 AND plor_nm이 제외 접두어로 시작하지 않는 것
mask = (df['plor_cd'] < 800000) & (~df['plor_nm'].str.startswith(tuple(excluded_prefixes)))

# 필터링
filtered_df = df[mask]

# 결과 저장
filtered_df.to_csv('표준코드/filtered_산지코드.csv', index=False, encoding='cp949')

In [3]:
import pandas as pd

# CSV 파일 경로
file_path = '표준코드/filtered_산지코드.csv'

# CSV 파일 읽기 (한글 인코딩 고려)
df = pd.read_csv(file_path, encoding='cp949')

# 축약형 -> 전체 행정구역 이름 매핑
region_map = {
    '강원': '강원도',
    '경기': '경기도',
    '경남': '경상남도',
    '경북': '경상북도',
    '광주': '광주광역시',
    '대구': '대구광역시',
    '대전': '대전광역시',
    '부산': '부산광역시',
    '서울': '서울특별시',
    '세종': '세종특별자치시',
    '울산': '울산광역시',
    '인천': '인천광역시',
    '전남': '전라남도',
    '전북': '전라북도',
    '제주': '제주특별자치도',
    '충남': '충청남도',
    '충북': '충청북도'
}

# 안전한 교체 함수
def replace_region_name_safe(plor_nm):
    for short, full in region_map.items():
        if plor_nm.startswith(full):  # 이미 전체 이름이면 그대로
            return plor_nm
        elif plor_nm.startswith(short):  # 축약형이면 교체
            return full + plor_nm[len(short):]
    return plor_nm  # 매칭 안 되면 그대로

# 적용
df['plor_nm'] = df['plor_nm'].apply(replace_region_name_safe)

# 결과 저장 (선택사항)
df.to_csv('표준코드/updated_산지코드.csv', index=False, encoding='cp949')

In [4]:
import pandas as pd

# CSV 파일 경로
file_path = '표준코드/updated_산지코드.csv'

# CSV 파일 읽기 (한글 인코딩 고려)
df = pd.read_csv(file_path, encoding='cp949')

# 도 + 시/군까지만 추출 (예: '강원도 춘천시')
df['region_city'] = df['plor_nm'].str.extract(r'^([^\s]+ [^\s]+)')

# plor_cd를 정수형으로 변환
df['plor_cd'] = df['plor_cd'].astype(int)

# 연속된 코드 묶기
def condense_ranges(codes):
    codes = sorted(set(codes))
    ranges = []
    start = prev = codes[0]

    for code in codes[1:]:
        if code == prev + 1:
            prev = code
        else:
            if start == prev:
                ranges.append(f"{start}")
            else:
                ranges.append(f"{start}~{prev}")
            start = prev = code

    if start == prev:
        ranges.append(f"{start}")
    else:
        ranges.append(f"{start}~{prev}")

    return ','.join(ranges)

# 지역별 plor_cd 묶기
grouped = df.groupby('region_city')['plor_cd'].apply(condense_ranges).reset_index()
grouped.columns = ['region_city', 'plor_cd_range']

# 결과 확인 또는 저장
print(grouped.head())

# 저장 (선택)
grouped.to_csv('표준코드/지역별_plor_cd범위.csv', index=False, encoding='cp949')

  region_city                                      plor_cd_range
0     강원도 강릉시  210000,210003,210010,210020,210030,210040,2100...
1     강원도 고성군  219000,219700,219705,219800~219804,219810~2198...
2     강원도 동해시  240000,240010,240020,240030,240041~240044,2400...
3     강원도 삼척시  235900~235914,235920~235921,245000,245010,2450...
4     강원도 속초시  217000,217010,217020,217030,217040,217050,2170...


In [5]:
import pandas as pd
import re

# CSV 불러오기
df_standard = pd.read_csv("표준코드/표준코드_산지코드.csv", encoding="cp949")
df_updated = pd.read_csv("표준코드/updated_산지코드.csv", encoding="cp949")

# 숫자형 처리
df_standard["plor_cd"] = pd.to_numeric(df_standard["plor_cd"], errors="coerce")
df_updated["plor_cd"] = pd.to_numeric(df_updated["plor_cd"], errors="coerce")
df_standard = df_standard.dropna(subset=["plor_cd"])
df_updated = df_updated.dropna(subset=["plor_cd"])
df_standard["plor_cd"] = df_standard["plor_cd"].astype(int)
df_updated["plor_cd"] = df_updated["plor_cd"].astype(int)

# 필터링
df_filtered = df_standard[
    (~df_standard["plor_cd"].isin(df_updated["plor_cd"])) &
    (df_standard["plor_cd"] < 800000)
].copy()

# 행정명칭 매핑
city_map = {
    "서울": "서울특별시",
    "부산": "부산광역시",
    "대구": "대구광역시",
    "인천": "인천광역시",
    "광주": "광주광역시",
    "대전": "대전광역시",
    "울산": "울산광역시",
    "세종": "세종특별자치시",
    "제주": "제주특별자치도"
}

# 행정명칭 중복 접미사 제거 (전처리 단계)
df_filtered["plor_nm"] = df_filtered["plor_nm"].apply(
    lambda x: re.sub(r"(특별자치시|특별자치도|광역시|특별시)[시도]$", r"\1", x)
)

# 정제 적용
for short, full in city_map.items():
    if short == "세종":
        df_filtered["plor_nm"] = df_filtered["plor_nm"].apply(
            lambda x: re.sub(r"^세종(시)?(?!특별자치시)", full, x)
            if not x.startswith("세종특별자치시") else x
        )
    elif short == "제주":
        df_filtered["plor_nm"] = df_filtered["plor_nm"].apply(
            lambda x: re.sub(r"^제주(도)?(?!특별자치도)", full, x)
            if not x.startswith("제주특별자치도") else x
        )
    else:
        pattern = rf"^{short}(?!시|특별시|광역시|특별자치시)"
        df_filtered["plor_nm"] = df_filtered["plor_nm"].apply(
            lambda x: re.sub(pattern, full, x)
        )
    
# 결과 저장 (선택)
df_filtered.to_csv("표준코드/표준코드_광역시_산지코드.csv", index=False, encoding="cp949")

In [6]:
import pandas as pd

# CSV 파일 로드
df_base = pd.read_csv("표준코드/표준코드_산지코드.csv", encoding="cp949")
df_over1 = pd.read_csv("표준코드/표준코드_광역시_산지코드.csv", encoding="cp949")
df_over2 = pd.read_csv("표준코드/updated_산지코드.csv", encoding="cp949")

# 모든 DataFrame에서 plor_cd 정리
for df in [df_base, df_over1, df_over2]:
    df["plor_cd"] = pd.to_numeric(df["plor_cd"], errors="coerce")
    df.dropna(subset=["plor_cd"], inplace=True)
    df["plor_cd"] = df["plor_cd"].astype(int)

# 덮어쓰기 대상 병합 + plor_cd 중복 제거 (나중 값 우선)
df_override = pd.concat([df_over1, df_over2], ignore_index=True)
df_override = df_override.drop_duplicates(subset="plor_cd", keep="last")

# plor_cd 기준으로 df_base 덮어쓰기
df_base.set_index("plor_cd", inplace=True)
df_override.set_index("plor_cd", inplace=True)

df_combined = df_override.combine_first(df_base).reset_index()


# 'plor_cd' 열을 숫자형으로 변환 (오류 시 NaN 처리), 이후 정수형(Int64)으로 변환
df['plor_cd'] = pd.to_numeric(df['plor_cd'], errors='coerce').astype('Int64')

# 결과 저장 (선택)
df_combined.to_csv("표준코드/표준코드_덮어쓰기완료.csv", index=False, encoding="cp949")

In [7]:
import pandas as pd
import re

# CSV 불러오기
df = pd.read_csv("표준코드/표준코드_덮어쓰기완료.csv", encoding='cp949')

# 주소에서 시도명 또는 시도 + 시/군 추출
def extract_main_region(name):
    if pd.isna(name):
        return pd.NA
    
    # 특별시, 광역시, 특별자치시 → 시도명까지만 추출
    match_city_only = re.match(r'^(.*?(특별시|광역시|특별자치시))', name)
    if match_city_only:
        return match_city_only.group(1)
    
    # 도, 특별자치도 → 시도 + 시/군까지 추출
    match_province = re.match(r'^(.*?(도|특별자치도))\s+(\S+?(시|군))', name)
    if match_province:
        return f"{match_province.group(1)} {match_province.group(3)}"
    
    return name  # 매치되지 않으면 원본 반환

# 적용
df['plor_nm'] = df['plor_nm'].apply(extract_main_region)

# 저장
df.to_csv("표준코드/표준코드_정리완료.csv", index=False, encoding='cp949')

In [8]:
import pandas as pd
from tqdm import tqdm

# CSV 파일 로딩
df = pd.read_csv("표준코드/표준코드_정리완료.csv", encoding='cp949')

df['plor_cd'] = pd.to_numeric(df['plor_cd'], errors='coerce')

# 정렬
df = df.sort_values('plor_cd').reset_index(drop=True)

# 1. 800000 미만 처리
df_lt = df[df['plor_cd'] < 800000].reset_index(drop=True)

range_rows = []
i = 0
with tqdm(total=len(df_lt)) as pbar:
    while i < len(df_lt):
        start_cd = df_lt.loc[i, 'plor_cd']
        plor_nm = df_lt.loc[i, 'plor_nm']
        j = i

        # 같은 plor_nm 연속 구간
        while j + 1 < len(df_lt) and df_lt.loc[j + 1, 'plor_nm'] == plor_nm:
            j += 1

        # 종료값 결정
        if j + 1 < len(df_lt):
            end_cd = df_lt.loc[j + 1, 'plor_cd'] - 1
        else:
            end_cd = 799999  # 마지막 구간은 강제 고정

        # 범위 문자열
        code_range = f"{start_cd}" if start_cd == end_cd else f"{start_cd}~{end_cd}"

        range_rows.append({
            '코드범위': code_range,
            'plor_nm': plor_nm
        })

        tqdm.write(f"처리됨: {plor_nm} ({code_range})")
        pbar.update(j - i + 1)
        i = j + 1

# 2. 800000 이상 원본 처리
df_gte = df[df['plor_cd'] >= 800000].copy()
df_gte['코드범위'] = df_gte['plor_cd'].astype(str)
df_gte['plor_nm'] = df_gte['plor_nm']
df_gte = df_gte[['코드범위', 'plor_nm']]

# 3. 병합 및 저장
result_df = pd.concat([pd.DataFrame(range_rows), df_gte], ignore_index=True)

# 저장
result_df.to_csv("표준코드/표준코드_정리완료1.csv", index=False, encoding='cp949')

print("✅ 저장 완료: 표준코드_정리완료1.csv")

 24%|█████████████████▍                                                        | 7423/31566 [00:00<00:00, 59512.91it/s]

처리됨: 서울특별시 (100000~199999)
처리됨: 강원도 춘천시 (200000~208999)
처리됨: 강원도 화천군 (209000~209823)
처리됨: 강원도 춘천시 (209824~209829)
처리됨: 강원도 화천군 (209830~209837)
처리됨: 강원도 춘천시 (209838)
처리됨: 강원도 철원군 (209839)
처리됨: 강원도 화천군 (209840~209999)
처리됨: 강원도 강릉시 (210000~214999)
처리됨: 강원도 양양군 (215000~216999)
처리됨: 강원도 속초시 (217000~218999)
처리됨: 강원도 고성군 (219000~219999)
처리됨: 강원도 원주시 (220000~224999)
처리됨: 강원도 횡성군 (225000~229999)
처리됨: 강원도 영월군 (230000~231999)
처리됨: 강원도 평창군 (232000~232999)
처리됨: 강원도 정선군 (233000~233828)
처리됨: 강원도 영월군 (233829)
처리됨: 강원도 정선군 (233830~234999)
처리됨: 강원도 태백시 (235000~235899)
처리됨: 강원도 삼척시 (235900~239999)
처리됨: 강원도 동해시 (240000~244999)
처리됨: 강원도 삼척시 (245000~245798)
처리됨: 강원도 동해시 (245799)
처리됨: 강원도 삼척시 (245800~245898)
처리됨: 강원도 동해시 (245899)
처리됨: 강원도 삼척시 (245900~249999)
처리됨: 강원도 홍천군 (250000~250818)
처리됨: 강원도 인제군 (250819)
처리됨: 강원도 홍천군 (250820~250825)
처리됨: 강원도 춘천시 (250826~250829)
처리됨: 강원도 홍천군 (250830~250856)
처리됨: 강원도 춘천시 (250857~250859)
처리됨: 강원도 홍천군 (250860~251999)
처리됨: 강원도 인제군 (252000~254999)
처리됨: 강원도 양구군 (255000~268999)


 34%|████████████████████████▊                                                | 10749/31566 [00:00<00:00, 59512.91it/s]

처리됨: 대전광역시 (300000~311999)
처리됨: 충청남도 금산군 (312000~313999)
처리됨: 충청남도 공주시 (314000~319999)
처리됨: 충청남도 논산시 (320000~320999)
처리됨: 충청남도 계룡시 (321000~322999)
처리됨: 충청남도 부여군 (323000~324999)
처리됨: 충청남도 서천군 (325000~329999)
처리됨: 충청남도 천안시 (330000~335999)
처리됨: 충청남도 아산시 (336000~338999)
처리됨: 세종특별자치시 (339000~339699)
처리됨: 충청남도 연기군 (339700~339999)
처리됨: 충청남도 예산군 (340000~342999)
처리됨: 충청남도 당진시 (343000~344999)
처리됨: 충청남도 청양군 (345000~349999)
처리됨: 충청남도 홍성군 (350000~354999)
처리됨: 충청남도 보령시 (355000~355999)
처리됨: 충청남도 서산시 (356000~356999)
처리됨: 충청남도 태안군 (357000~359999)
처리됨: 충청북도 청주시 (360000~362999)
처리됨: 충청북도 청원군 (363000~364999)
처리됨: 충청북도 진천군 (365000~366999)
처리됨: 충청북도 괴산군 (367000~367701)
처리됨: 충청북도 청주시 (367702~367799)
처리됨: 충청북도 괴산군 (367800~367999)
처리됨: 충청북도 증평군 (368000~368999)
처리됨: 충청북도 음성군 (369000~369999)
처리됨: 충청북도 영동군 (370000~372999)
처리됨: 충청북도 옥천군 (373000~375999)
처리됨: 충청북도 보은군 (376000~379999)
처리됨: 충청북도 충주시 (380000~389999)
처리됨: 충청북도 제천시 (390000~394999)


처리됨: 충청북도 단양군 (395000~399999)
처리됨: 인천광역시 (400000~409999)
처리됨: 경기도 고양시 (410000~412999)
처리됨: 경기도 파주시 (413000~414999)
처리됨: 경기도 김포시 (415000~416999)
처리됨: 인천광역시 (417000~419999)
처리됨: 경기도 부천시 (420000~422999)
처리됨: 경기도 광명시 (423000~424999)
처리됨: 경기도 안산시 (425000~426999)
처리됨: 경기도 과천시 (427000~428999)
처리됨: 경기도 시흥시 (429000~429999)
처리됨: 경기도 안양시 (430000~434999)
처리됨: 경기도 군포시 (435000~436958)
처리됨: 경기도 성남시 (436959~436999)
처리됨: 경기도 의왕시 (437000~439999)
처리됨: 경기도 수원시 (440000~444999)
처리됨: 경기도 화성시 (445000~445999)
처리됨: 경기도 용인시 (446000~446999)


...

처리됨: 전라남도 완도군 (536935~536939)
처리됨: 전라남도 해남군 (536940~536999)
처리됨: 전라남도 완도군 (537000~538999)
처리됨: 전라남도 진도군 (539000~539999)
처리됨: 전라남도 순천시 (540000~541999)
처리됨: 전라남도 구례군 (542000~544999)
처리됨: 전라남도 광양시 (545000~545999)
처리됨: 전라남도 보성군 (546000~547999)
처리됨: 전라남도 고흥군 (548000~548989)
처리됨: 전라남도 완도군 (548990~549999)
처리됨: 전라남도 여수시 (550000~559999)
처리됨: 전라북도 전주시 (560000~564999)
처리됨: 전라북도 완주군 (565000~565999)
처리됨: 전라북도 임실군 (566000~566999)
처리됨: 전라북도 진안군 (567000~567999)
처리됨: 전라북도 무주군 (568000~569999)
처리됨: 전라북도 익산시 (570000~572999)
처리됨: 전라북도 군산시 (573000~575999)
처리됨: 전라북도 김제시 (576000~578999)
처리됨: 전라북도 부안군 (579000~579893)
처리됨: 전라북도 고창군 (579894~579909)
처리됨: 전라북도 부안군 (579910~579999)
처리됨: 전라북도 정읍시 (580000~584999)
처리됨: 전라북도 고창군 (585000~589999)
처리됨: 전라북도 남원시 (590000~594999)
처리됨: 전라북도 순창군 (595000~596999)
처리됨: 전라북도 장수군 (597000~599999)


 89%|████████████████████████████████████████████████████████████████▉        | 28089/31566 [00:01<00:00, 19006.52it/s]

처리됨: 부산광역시 (600000~620999)
처리됨: 경상남도 김해시 (621000~625999)
처리됨: 경상남도 양산시 (626000~626999)
처리됨: 경상남도 밀양시 (627000~629999)
처리됨: 경상남도 창원시 (630000~630009)
처리됨: 경상남도 마산시 (630010~634999)
처리됨: 경상남도 창녕군 (635000~635999)
처리됨: 경상남도 의령군 (636000~636999)
처리됨: 경상남도 함안군 (637000~637999)
처리됨: 경상남도 고성군 (638000~640999)
처리됨: 경상남도 창원시 (641000~644999)
처리됨: 경상남도 진해시 (645000~649999)
처리됨: 경상남도 통영시 (650000~655999)
처리됨: 경상남도 거제시 (656000~659999)
처리됨: 경상남도 진주시 (660000~663999)
처리됨: 경상남도 사천시 (664000~665999)
처리됨: 경상남도 산청군 (666000~666999)
처리됨: 경상남도 하동군 (667000~667999)
처리됨: 경상남도 남해군 (668000~669999)
처리됨: 경상남도 거창군 (670000~675999)
처리됨: 경상남도 함양군 (676000~677999)
처리됨: 경상남도 합천군 (678000~679999)
처리됨: 울산광역시 (680000~689999)
처리됨: 제주특별자치도 제주시 (690000~695749)
처리됨: 제주특별자치도 북제주군 (695750~695790)
처리됨: 제주특별자치도 제주시 (695791~695799)
처리됨: 제주특별자치도 북제주군 (695800~695899)
처리됨: 제주특별자치도 제주시 (695900~696999)
처리됨: 제주특별자치도 서귀포시 (697000~699799)
처리됨: 제주특별자치도 남제주군 (699800~699899)
처리됨: 제주특별자치도 서귀포시 (699900~699999)


100%|█████████████████████████████████████████████████████████████████████████| 31566/31566 [00:01<00:00, 20092.69it/s]


처리됨: 대구광역시 (700000~711999)
처리됨: 경상북도 경산시 (712000~713999)
처리됨: 경상북도 청도군 (714000~715999)
처리됨: 경상북도 군위군 (716000~716999)
처리됨: 경상북도 고령군 (717000~717999)
처리됨: 경상북도 칠곡군 (718000~718999)
처리됨: 경상북도 성주군 (719000~729999)
처리됨: 경상북도 구미시 (730000~739999)
처리됨: 경상북도 김천시 (740000~741999)
처리됨: 경상북도 상주시 (742000~744999)
처리됨: 경상북도 문경시 (745000~749999)
처리됨: 경상북도 영주시 (750000~754999)
처리됨: 경상북도 봉화군 (755000~756999)
처리됨: 경상북도 예천군 (757000~759999)
처리됨: 경상북도 안동시 (760000~762999)
처리됨: 경상북도 청송군 (763000~763999)
처리됨: 경상북도 영양군 (764000~765999)
처리됨: 경상북도 영덕군 (766000~766999)
처리됨: 경상북도 울진군 (767000~768999)
처리됨: 경상북도 의성군 (769000~769999)
처리됨: 경상북도 영천시 (770000~779999)
처리됨: 경상북도 경주시 (780000~789999)
처리됨: 경상북도 포항시 (790000~798999)
처리됨: 경상북도 울릉군 (799000~799999)
✅ 저장 완료: 표준코드_정리완료1.csv


In [17]:
import pandas as pd

# 파일 경로
file_path = "표준코드/표준코드_정리완료1.csv"

# CSV 파일 불러오기 (인코딩: cp949)
df = pd.read_csv(file_path, encoding='cp949')

# '코드범위' 컬럼명을 'plor_cd'로 변경
df = df.rename(columns={'코드범위': 'plor_cd'})

# 고유한 plor_nm 기준으로 new_cd, new_nm 생성
df_mapping = df.drop_duplicates(subset='plor_nm').reset_index(drop=True)
df_mapping['new_cd'] = range(1000, 1000 + len(df_mapping))
df_mapping['new_nm'] = df_mapping['plor_nm']

# 전체 원본 데이터에 매핑값 병합
df_full = df.merge(df_mapping[['plor_nm', 'new_cd', 'new_nm']], on='plor_nm', how='left')

# 결과 확인
print(df_full.head())

# 저장 (원할 경우)
df_full.to_csv("표준코드/산지코드_new.csv", index=False, encoding='cp949')

         plor_cd  plor_nm  new_cd   new_nm
0  100000~199999    서울특별시    1000    서울특별시
1  200000~208999  강원도 춘천시    1001  강원도 춘천시
2  209000~209823  강원도 화천군    1002  강원도 화천군
3  209824~209829  강원도 춘천시    1001  강원도 춘천시
4  209830~209837  강원도 화천군    1002  강원도 화천군


In [18]:
import pandas as pd

# 파일 경로
file_path = "표준코드/산지코드_new.csv"

# CSV 파일 불러오기 (인코딩 cp949)
df = pd.read_csv(file_path, encoding='cp949')

# 컬럼명 변경
df_renamed = df.rename(columns={
    'plor_cd': '산지코드',
    'plor_nm': '산지이름',
    'new_cd': '직팜산지코드',
    'new_nm': '직팜산지이름'
})

# 결과 확인
print(df_renamed.head())

# 저장 (필요시)
df_renamed.to_csv("표준코드/산지코드_직팜.csv", index=False, encoding='cp949')

            산지코드     산지이름  직팜산지코드   직팜산지이름
0  100000~199999    서울특별시    1000    서울특별시
1  200000~208999  강원도 춘천시    1001  강원도 춘천시
2  209000~209823  강원도 화천군    1002  강원도 화천군
3  209824~209829  강원도 춘천시    1001  강원도 춘천시
4  209830~209837  강원도 화천군    1002  강원도 화천군
